In [1]:
import numpy as np
import pandas as pd
from keras import models
from keras import layers, optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

df = pd.read_csv("pydf.csv")

print(df.head(10))


Using TensorFlow backend.


   Unnamed: 0         Y        X1        X2        X3        X4        X5  \
0           1 -0.008751 -0.144204  0.073449  0.172874 -0.354829 -0.044645   
1           2 -0.016501  0.400409  0.105899 -0.492753  0.163660  0.131203   
2           3  0.100915 -0.387580 -0.178040  0.284180 -0.030406 -0.116767   
3           4  0.095073 -0.219716 -0.014305  0.351343  0.268202  0.105462   
4           5  0.006130  0.282264  0.149557 -0.070823  0.273824 -0.309345   
5           6 -0.083144  0.079929 -0.274689  0.492370 -0.137467  0.162975   
6           7 -0.014818  0.009452  0.162206 -0.156530 -0.128760  0.053469   
7           8  0.087826  0.257970  0.218494  0.322699  0.133582  0.312357   
8           9 -0.019344  0.095346 -0.037835  0.152475  0.042091 -0.135404   
9          10 -0.004574 -0.331753  0.045588  0.016681  0.365589  0.501461   

         X6        X7        X8  ...       X93       X94       X95       X96  \
0 -0.249352 -0.039039  0.221268  ...  0.119313 -0.093472 -0.019785  0.01

In [ ]:
train_T = round(max(df['time']) / 2)
val_T = round(max(df['time']))*3/4
train = df[df['time'] <= train_T]
val = df[(train_T < df['time']) & (df['time'] <= val_T)]
test = df[df['time'] > val_T]
train.drop(['time', 'id'], axis = 1, inplace = True)
val.drop(['time', 'id'], axis = 1, inplace = True)
test.drop(['time', 'id'], axis = 1, inplace = True)

# Split into y and x
x_train = train.loc[:,'X1':]
y_train = train.loc[:,'Y']
x_val = val.loc[:,'X1':]
y_val = val.loc[:,'Y']
x_test = test.loc[:,'X1':]
y_test = test.loc[:,'Y']


np.random.seed(0)

number_of_features = len(x_train.iloc[1]) 

# Start neural network
network = models.Sequential()

# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=16, activation='relu', input_shape=(number_of_features,)))
network.add(layers.normalization.BatchNormalization())

# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=8, activation='relu'))
network.add(layers.normalization.BatchNormalization())

# Add fully connected layer with a sigmoid activation function
network.add(layers.Dense(units=1, activation='linear'))

# Compile neural network
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
network.compile(loss='mean_squared_error', # RSS
                optimizer='sgd') # Stochastic gradient descent

# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

# Train neural network
history = network.fit(x_train, # Features
                      y_train, # Target vector
                      epochs=100, # Number of epochs
                      callbacks=callbacks, # Early stopping
                      verbose=0, # Print description after each epoch
                      batch_size=32, # Number of observations per batch
                      validation_data=(x_val, y_val)) # Data for evaluation

# Predict on test set
prediction = network.predict(x_test, batch_size=128)
mse = np.mean(np.square(prediction[:,0] - y_test))
total = np.mean(np.square(y_test - np.mean(y_test)))
R2 = 1 - mse/total
print(R2)

Instructions for updating:
Colocations handled automatically by placer.


/Users/alexanderbech/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Instructions for updating:
Use tf.cast instead.
